# Sentiment Analysis and Stock Prediction Model

This project made by Divij Jasuja and Pranay Raturi for Data Mining Project

In [6]:
%pip install torch torchmetrics


  Using cached torch-2.3.0-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached torchmetrics-1.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.0-cp311-cp311-win_amd64.whl (159.8 MB)
Using cached torchmetrics-1.3.2-py3-none-any.whl (841 kB)
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
Using cached tbb-2021.12.0-py3-none-win_amd64.whl (286 k

In [1]:
# importing libraries
import torch
from torch import nn
from torch import optim
import math
import pandas as pd
import datetime as dt

## Sentiment Analysis using roberta

In [4]:
%pip install transformers

  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.4.28-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.3-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl (144 kB)
Using cached regex-2024.4.28-cp311-cp311-win_amd64.whl (268 kB)
Using cached safetensors-0.4.3-cp311-none-win_amd64.whl (287 kB)
Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl (2.2 MB)
Using cached filelock-3.14.0-py3-none-any.whl (12 kB

In [2]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline
nlp = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Use the pipeline to analyze the sentiment of a text
text = "nvidia is a good company"
result = nlp(text)

# The result is a list of dictionaries. Each dictionary contains the 'label' and 'score'.
# 'label' is the sentiment (POSITIVE or NEGATIVE), and 'score' is the confidence.
sentiment = result[0]['label']
confidence = result[0]['score']

print(f"The sentiment is {sentiment} with a confidence of {confidence}")

d:\projects\sentiment analysis and stock prediction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The sentiment is LABEL_2 with a confidence of 0.9292691349983215


In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [4]:
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
text = "stocks are ok"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores.argmax()

2

In [6]:
import pandas as pd
tweets = pd.read_csv('twitter_data/NVDA_final-tweets')
tweets.head()

,Unnamed: 0,Tweet_ID,Time_Created,Text,Likes,Retweets,cleaned
0,0,1744146496243822931,2024-01-07,"@SelfMadeMastery Best: $NVDA, $CRWD, $META, $T...",1,0,best nvda crwd meta tsla bad enph use oppurtun...
1,1,1744146280576926128,2024-01-07,Most Notable #Earnings Week of JAN 8th\n\n◦ Mo...,0,0,notable earnings week jan mon accd tues tlry a...
2,2,1744145386850422822,2024-01-07,🇺🇸 U.S. ECONOMIC DATA 2ND WEEK\n\nTHURS.\n◦ U....,0,0,flag united state economic data week thurs cpi...
3,3,1744145096592007411,2024-01-07,$MARA We nailed this play. I kept on hammering...,3,0,mara nail play kept hammer risk reward hence l...
4,4,1743200305033175350,2024-01-05,$MARA Scaled out of this position yesterday\n\...,1,0,mara scale position yesterday last time specul...


In [17]:
def sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt',truncation=True, max_length=512)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores.argmax()

In [19]:
from tqdm import tqdm
tqdm.pandas()
tweets['cleaned'] = tweets['cleaned'].fillna("")
tweets['sentiment'] = tweets['cleaned'].progress_apply(lambda x: sentiment_analysis(x))

100%|██████████| 269199/269199 [5:52:34<00:00, 12.73it/s]  


In [20]:
tweets.to_csv('analysed_tweets.csv')

In [9]:
import pandas as pd
pd.read_csv('analysed_tweets.csv').loc[:,'sentiment'].value_counts()

scentiment
1    196795
2     46757
0     25647
Name: count, dtype: int64

## stock prediction using LSTM

In [ ]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [ ]:
import torch
import torch.nn as nn
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)